In [1]:
import json
from glob import glob
import fnmatch
from multiprocessing import Pool
from pathlib import Path
import os, itertools
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from pickle import dumps, loads, dump, load

from stexls import *

In [2]:
%rm /tmp/linker.bin

rm: cannot remove '/tmp/linker.bin': No such file or directory


In [3]:
root = Path.home()/'MathHub'

In [ ]:
try:
    with open('/tmp/linker.bin', 'rb') as fd:
        linker = load(fd)
except Exception as e:
    print(e)
    linker = Linker(root=root, file_pattern='**/*.tex')
    print(linker.update(progress=tqdm, use_multiprocessing=True))
    with open('/tmp/linker.bin', 'w+b') as fd:
        dump(linker, fd)

In [ ]:
%ls -ahl /tmp/linker.bin

In [4]:
path = root / Path('MiKoMH/AI/source/complexity/en/recap.tex')

In [ ]:
linker.view_import_graph(path)

In [ ]:
linker.update(tqdm)

In [ ]:
for object in linker.objects[path]:
    print(linker.links[object].format())
    print()
    print('-'*80)
    print()